In [1]:
import re
import os
import string
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight
from sentence_transformers import SentenceTransformer

from cleanlab.classification import CleanLearning
from rich import print

pd.set_option('display.max_colwidth', None)

In [2]:
data_path = '../../data/result/train_with_topic_df_majority_vote_gpt4o_preferred.csv'

In [3]:
data = pd.read_csv(data_path)
data.rename(columns={'topic_id': 'old_topic_id'}, inplace=True)

In [4]:
data.head()

,column,text,old_topic_id,topic_name,topic_llm,majority_vote,tie
0,like,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه للعمل والقضايا الواقعية لتحليل الأدلة الجنائية التقنية,3,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري,False
1,improve_course,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس الأكاديمي,3,موازنة الجزء العملي مع الجزء النظري,طريقة تدريس و تقديم المادة,طريقة تدريس و تقديم المادة,False
2,like,أكثر ما أعجبني: the content,0,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر,False
3,improve_course,اقتراحاتي للتحسين: provide more new books,3,موازنة الجزء العملي مع الجزء النظري,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر,False
4,improve_course,اقتراحاتي للتحسين: زيادة ساعات العملي,8,الوقت و الجدول,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري,False


In [5]:
data = data.drop(columns=['old_topic_id', 'column', 'topic_llm', 'tie', 'topic_name'])

In [6]:
data = data.rename(columns={'majority_vote': 'topic_name'})

In [7]:
data.head()

,text,topic_name
0,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه للعمل والقضايا الواقعية لتحليل الأدلة الجنائية التقنية,موازنة الجزء العملي مع الجزء النظري
1,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس الأكاديمي,طريقة تدريس و تقديم المادة
2,أكثر ما أعجبني: the content,محتوى ومعلومات المقرر
3,اقتراحاتي للتحسين: provide more new books,محتوى ومعلومات المقرر
4,اقتراحاتي للتحسين: زيادة ساعات العملي,موازنة الجزء العملي مع الجزء النظري


In [8]:
# prev_iteration_path = '../../data/cleansed_data/topics/first_iteration/label_issues_topics_majority_vote_first_iteration_logistic_regression(solved).csv'
# prev_iteration = pd.read_csv(prev_iteration_path)

In [9]:
# prev_iteration.head()

In [10]:
# data['topic_name'] = prev_iteration['updated_label']

In [11]:
# data['topic_name'] = data['topic_name'].apply(lambda x: "غير محدد" if x == 'None' else x)

In [12]:
def remove_prefix(x):
    prefix_list = [
        "أكثر ما أعجبني:",
        "أكثر ما لم يعجبني:",
        "اقتراحاتي للتحسين:",
        "اقتراحاتي للإضافة:"
    ]

    for p in prefix_list:
        x = x.replace(p, '').strip()

    return x

In [13]:
# data['text'] = data['text'].apply(remove_prefix)

In [14]:
data.head(10)

,text,topic_name
0,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه للعمل والقضايا الواقعية لتحليل الأدلة الجنائية التقنية,موازنة الجزء العملي مع الجزء النظري
1,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس الأكاديمي,طريقة تدريس و تقديم المادة
2,أكثر ما أعجبني: the content,محتوى ومعلومات المقرر
3,اقتراحاتي للتحسين: provide more new books,محتوى ومعلومات المقرر
4,اقتراحاتي للتحسين: زيادة ساعات العملي,موازنة الجزء العملي مع الجزء النظري
5,أكثر ما أعجبني: شرح البروفيسور ممتاز ومعرفة شاملة عن المادة,دكتور المقرر
6,أكثر ما أعجبني: تفاعل و تعاون الأستاذ اضافه الكثير من المعلومات الهامه جدا فيما يتعلق بالتخصص,دكتور المقرر
7,اقتراحاتي للإضافة: د. اعتماد من الدكاتره المحترمين لكن للاسف بنفس الوقت من الدكاتره التي لا أرغب في أخذ اي مقرر معها نهائي لانها مع احترامي لها عشوائية بالتدريس لاتلتزم بالخطة فهي دايم تضغطنا في نهاية الفصل بجميع التكاليف والواجبات مع العلمانناكننا نطلب منها إرسال الواجبات بعد نهاية كل فصل حتى يكون لدينا متسع من الوقت لكن بدون جدوى كمااننا لانعرف درجاتنا الا في نهاية الفصل توزيعنا في البيبرات بمجموعات قد يتضمن طالبات لايعملون عند عرض بعض المراجع الخارجيه عليها كانت ترفض وتقول ان بيني وبينكم الكتاب فقط عند حل الواجبات معها كانت تستخدم المانيوال ولما نستفسر عن طريقة حل بعض الأسئلة كانت لاتعرف أتمنى عدم تدريسها لاي مقرر شاكره لكم,دكتور المقرر
8,أكثر ما لم يعجبني: noting,غير محدد
9,أكثر ما لم يعجبني: التعمق في جزء الإحصاء بشكل كبير جدا,محتوى ومعلومات المقرر


In [15]:
text = data['text'].values
labels = data['topic_name'].values

In [16]:
# test_size = 0.2
# raw_train_texts, raw_test_texts, raw_train_labels, raw_test_labels = train_test_split(text, labels, test_size=test_size)
raw_train_texts, raw_train_labels = text.copy(), labels.copy()

In [17]:
num_classes = len(set(raw_train_labels))

print(f"This dataset has {num_classes} classes.")
print(f"Classes:\n{set(raw_train_labels)}")

This dataset has 9 classes.

Classes:
{'سوق العمل', 'المتطلبات و المهام و الدرجات', 'دكتور المقرر', 'محتوى ومعلومات المقرر', 'غير محدد', 'الجانب البحثي و
الأوراق البحثية', 'طريقة تدريس و تقديم المادة', 'الوقت و الجدول', 'موازنة الجزء العملي مع الجزء النظري'}

In [18]:
label_encoder = LabelEncoder()
label_encoder.fit(raw_train_labels)

train_labels = label_encoder.transform(raw_train_labels)
# test_labels = label_encoder.transform(raw_test_labels)

In [19]:
train_labels[:2]

array([8, 5])

In [20]:
model_path = 'sentence-transformers/use-cmlm-multilingual'
embedding_model = SentenceTransformer(model_path)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [21]:
train_texts = embedding_model.encode(raw_train_texts, show_progress_bar=True)
# test_texts = embedding_model.encode(raw_test_texts)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [22]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)

In [23]:
type(class_weights)

numpy.ndarray

In [24]:
class_weights_dict = {i: w for i, w in enumerate(class_weights)}

In [25]:
print(class_weights_dict)

{
    0: 3.019047619047619,
    1: 1.427927927927928,
    2: 2.7094017094017095,
    3: 0.6003787878787878,
    4: 3.4086021505376345,
    5: 1.447488584474886,
    6: 0.806615776081425,
    7: 0.46755162241887904,
    8: 0.6365461847389559
}

In [26]:
print(label_encoder.inverse_transform(list(range(9))))

['الجانب البحثي و الأوراق البحثية' 'المتطلبات و المهام و الدرجات'
 'الوقت و الجدول' 'دكتور المقرر' 'سوق العمل' 'طريقة تدريس و تقديم المادة'
 'غير محدد' 'محتوى ومعلومات المقرر' 'موازنة الجزء العملي مع الجزء النظري']

In [27]:
epochs = 500
model = LogisticRegression(max_iter=epochs, class_weight=class_weights_dict)

In [28]:
cv_n_folds = 5 
random_state = 42
cl = CleanLearning(model, cv_n_folds=cv_n_folds, verbose=True, seed=random_state)

In [29]:
label_issues = cl.find_label_issues(X=train_texts, labels=train_labels)

2024-08-05 04:06:50.601365: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 04:06:50.601444: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 04:06:50.602874: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 04:06:50.611805: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-05 04:06:52.005756: W tensorflow/compiler/tf2

Computing out of sample predicted probabilities via 5-fold cross validation. May take a while ...
Using predicted probabilities to identify label issues ...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Identified 93 examples with label issues.


In [30]:
len(label_issues[label_issues['is_label_issue']==True])

93

In [31]:
len(label_issues)

951

In [32]:
label_issues.head()

,is_label_issue,label_quality,given_label,predicted_label
0,False,0.468037,8,8
1,False,0.224766,5,8
2,False,0.541209,7,7
3,False,0.160496,7,6
4,False,0.212089,8,2


In [33]:
label_issues['text'] = text

In [34]:
# label_issues = label_issues[label_issues["is_label_issue"] == True]
# label_issues.sort_values(by='label_quality', ascending=True, inplace=True)

In [35]:
label_issues['given_label_str'] = label_encoder.inverse_transform(label_issues['given_label'].values)
label_issues['predicted_label_str'] = label_encoder.inverse_transform(label_issues['predicted_label'].values)

In [36]:
label_issues

,is_label_issue,label_quality,given_label,predicted_label,text,given_label_str,predicted_label_str
0,False,0.468037,8,8,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه للعمل والقضايا الواقعية لتحليل الأدلة الجنائية التقنية,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري
1,False,0.224766,5,8,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس الأكاديمي,طريقة تدريس و تقديم المادة,موازنة الجزء العملي مع الجزء النظري
2,False,0.541209,7,7,أكثر ما أعجبني: the content,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر
3,False,0.160496,7,6,اقتراحاتي للتحسين: provide more new books,محتوى ومعلومات المقرر,غير محدد
4,False,0.212089,8,2,اقتراحاتي للتحسين: زيادة ساعات العملي,موازنة الجزء العملي مع الجزء النظري,الوقت و الجدول
...,...,...,...,...,...,...,...
946,False,0.396413,6,6,اقتراحاتي للتحسين: الشكر الجزيل لكل من ساهم في إنجاح هذ البرنامج,غير محدد,غير محدد
947,False,0.620421,2,2,اقتراحاتي للتحسين: تحسين وقت المحاضرات,الوقت و الجدول,الوقت و الجدول
948,False,0.829286,6,6,اقتراحاتي للإضافة: nothing i can think of,غير محدد,غير محدد
949,False,0.603333,1,1,أكثر ما لم يعجبني: أسئلة الواجب ليس واضح,المتطلبات و المهام و الدرجات,المتطلبات و المهام و الدرجات


In [37]:
len(label_issues[label_issues['is_label_issue']==True])

93

In [39]:
data_path

'../../data/result/train_with_topic_df_majority_vote_gpt4o_preferred.csv'

In [40]:
save_path = '../../data/result/label_issues_topics_majority_vote_first_iteration_logistic_regression.csv'
if os.path.exists(save_path):
    print('Path already exists!')

else:
    label_issues.to_csv(save_path, index=False)

In [42]:
# topics_save_path = '../../data/result/train_with_topic_df_majority_vote_gpt4o_preferred_after_first_iteration.csv'

# data.to_csv(topics_save_path, index=False)